In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-summary/news_summary_more.csv
/kaggle/input/news-summary/news_summary.csv


In [1]:
conda create -n myenv python "numpy>=1.22.3,<1.23.0" scipy

/bin/bash: 1.23.0: No such file or directory

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install dash==1.20.0 

In [6]:
!pip install --upgrade pip
!pip install --root-user-action=ignore requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [10]:
!pip install simplet5 -q 

In [11]:
df = pd.read_csv("../input/news-summary/news_summary.csv", encoding='latin-1', usecols=['headlines', 'text'])

In [12]:
df.head()

,headlines,text
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...


In [13]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"headlines":"target_text", "text":"source_text"})
df = df[['source_text', 'target_text']]

In [14]:
df.head()

,source_text,target_text
0,The Administration of Union Territory Daman an...,Daman & Diu revokes mandatory Rakshabandhan in...
1,Malaika Arora slammed an Instagram user who tr...,Malaika slams user who trolled her for 'divorc...
2,The Indira Gandhi Institute of Medical Science...,'Virgin' now corrected to 'Unmarried' in IGIMS...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Aaj aapne pakad liya: LeT man Dujana before be...
4,Hotels in Maharashtra will train their staff t...,Hotel staff to get training to spot signs of s...


In [15]:
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: The Administration of Union Territo...,Daman & Diu revokes mandatory Rakshabandhan in...
1,summarize: Malaika Arora slammed an Instagram ...,Malaika slams user who trolled her for 'divorc...
2,summarize: The Indira Gandhi Institute of Medi...,'Virgin' now corrected to 'Unmarried' in IGIMS...
3,summarize: Lashkar-e-Taiba's Kashmir commander...,Aaj aapne pakad liya: LeT man Dujana before be...
4,summarize: Hotels in Maharashtra will train th...,Hotel staff to get training to spot signs of s...
...,...,...
4509,summarize: Fruit juice concentrate maker Rasna...,Rasna seeking ?250 cr revenue from snack categ...
4510,summarize: Former Indian cricketer Sachin Tend...,Sachin attends Rajya Sabha after questions on ...
4511,"summarize: Aamir Khan, while talking about rea...",Shouldn't rob their childhood: Aamir on kids r...
4512,summarize: The Maharashtra government has init...,"Asha Bhosle gets ?53,000 power bill for unused..."


In [16]:
train_df, test_df = train_test_split(df, test_size=0.3)
train_df.shape, test_df.shape

((3159, 2), (1355, 2))

In [18]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [28]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=512, 
            target_max_token_len=100, 
            batch_size=8, max_epochs=5, use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [29]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-0.4144-val-loss-1.4778
simplet5-epoch-0-train-loss-1.0477-val-loss-1.2687
simplet5-epoch-1-train-loss-0.3607-val-loss-1.5967
simplet5-epoch-1-train-loss-1.0041-val-loss-1.2814
simplet5-epoch-2-train-loss-0.2924-val-loss-1.703
simplet5-epoch-2-train-loss-0.7977-val-loss-1.3159
simplet5-epoch-3-train-loss-0.2423-val-loss-1.7435
simplet5-epoch-3-train-loss-0.6457-val-loss-1.3693
simplet5-epoch-4-train-loss-0.2063-val-loss-1.8198
simplet5-epoch-4-train-loss-0.5293-val-loss-1.4517


In [30]:
import os
os.chdir(r'/kaggle/working')

In [31]:
%cd /kaggle/working
from IPython.display import FileLinks
FileLinks(r'outputs')

/kaggle/working


outputs/simplet5-epoch-0-train-loss-0.4144-val-loss-1.4778/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-0-train-loss-1.0477-val-loss-1.2687/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-1-train-loss-0.3607-val-loss-1.5967/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-1-train-loss-1.0041-val-loss-1.2814/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-2-train-loss-0.2924-val-loss-1.703/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-2-train-loss-0.7977-val-loss-1.3159/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-3-train-loss-0.2423-val-loss-1.7435/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-3-train-loss-0.6457-val-loss-1.3693/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-4-train-loss-0.2063-val-loss-1.8198/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-4-train-loss-0.5293-val-loss-1.4517/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json

In [32]:
model.load_model("t5","outputs/simplet5-epoch-4-train-loss-0.2063-val-loss-1.8198", use_gpu=True)

In [33]:
text_to_summarize="""summarize: Twitter’s interim resident grievance officer for India has stepped down, leaving the micro-blogging site without a grievance official as mandated by the new IT rules to address complaints from Indian subscribers, according to a source.

The source said that Dharmendra Chatur, who was recently appointed as interim resident grievance officer for India by Twitter, has quit from the post.

The social media company’s website no longer displays his name, as required under Information Technology (Intermediary Guidelines and Digital Media Ethics Code) Rules 2021.

Twitter declined to comment on the development.

The development comes at a time when the micro-blogging platform has been engaged in a tussle with the Indian government over the new social media rules. The government has slammed Twitter for deliberate defiance and failure to comply with the country’s new IT rules.
"""
model.predict(text_to_summarize)

["Twitter's interim resident grievance officer steps down"]

In [36]:
text = """ Egypt’s Foreign Minister Sameh Shoukry met on Monday Chadian President Mohamed Idriss Deby, at the beginning of his two-stop tour in Africa.

Ambassador Ahmed Abu Zeid, the spokesperson of the Ministry of Foreign Affairs, said that Shoukry handed over a letter from President Abdel Fattah Al-Sisi to President Deby addressing, on one side, ways to support and strengthen bilateral relations between the two countries in various fields, and on the other side, consultation and coordination on the ongoing developments in Sudan and the role of Chad and Sudan’s other neighbouring countries in supporting ceasefire efforts and dealing with the humanitarian crisis faced by the brotherly Sudanese people as a result of the ongoing fighting.

 During the meeting, the Chadian president expressed his deep appreciation for the initiative of President AL- Sisi to send the Foreign Minister to Chad at this important time when the brotherly Sudan is facing a very serious security and political challenge. 

He expressed Chad’s aspiration to coordinate with Egypt to encourage the Sudanese parties to establish and adhere to a comprehensive ceasefire that allows the delivery of humanitarian aid and relief, as well as to allow for the launch of a constructive dialogue to resolve differences between the Sudanese parties.

 For his part, Minister Shoukry reviewed the efforts and contacts made by Egypt since the beginning of the crisis, at the political level through direct communication with the parties to the conflict to encourage them to cease fire, address the causes of the dispute through dialogue, and at the humanitarian level by emphasizing the priority of injecting blood and sparing the Sudanese people the ravages of that internecine war.

 Shoukry also said that Egypt opened its arms to receive those fleeing from the hell of the military confrontations by the Sudanese brothers in numbers of more than 60,000 people.

He referred to the great humanitarian tragedy caused by this crisis, which is directly affected by the neighboring countries of Sudan, which requires the intensification of consultation and coordination mechanisms. 

 The two sides agreed to communicate closely at various levels during the next stage, and coordinate positions at the regional and international levels, in order to stop the ongoing war and preserve the integrity, unity and sovereignty of Sudan.

In the same context, Saudi Foreign Minister Faisal bin Farhan discussed with the high representative of the EU for Foreign Affairs and security policy Josep Borrell the latest developments in the situation in Sudan.

During a telephone call on Monday, the two sides reviewed the latest developments of the Saudi – American initiative to host representatives of the Sudanese armed forces and the Rapid Support Forces (RSF) in Jeddah, which aims to create the necessary ground for dialogue to reduce the level of tensions there.

According to a Saudi diplomat on Monday, the ceasefire negotiations between the warring parties in Sudan and held in Saudi Arabia have not made “significant progress,” which weakens hopes for a quick cessation of hostilities.

 """

model.predict(text)

["Egypt's Foreign Minister meets with Chad Prez Deby"]